# **INTRODUCTION**
#### In this ambitious Capstone project, we explore the American auto market, analyzing pricing factors using data and advanced tools to uncover hidden insights for XYZ Motors' success.
### Libraries To Be Used:
- #### Pandas 
- #### Numpy

                             Task 1: Loading and Initial Data Exploration:

In [ ]:
## Importing the libraries
import pandas as pd 
import numpy as np 

# Loading the data
df_unclean = pd.read_csv(r"C:\Users\TANAY_MAHARANA\Documents\Python scripts\csv,text etc files for practice\car-prices_unclean - car-prices_unclean.csv.csv")
print(df_unclean["price"].median())
print("First Few Rows of the Dataset:\n",df_unclean.head())  ## printing the initial 5rows            
print()
print("Dimensions of the Dataset:\n",df_unclean.shape)  ## checking dataset dimensions                 
print()
print("Summary of the Dataset:")  
print(df_unclean.info()) ## checking the summary of the dataset          
print()
print("Missing Values in the Dataset:\n",df_unclean.isnull().sum())  ## checking for null values in the dataset
print()
print("Duplicate values in the dataset:\n",df_unclean.duplicated().sum())  ## checking for duplicate value sin the data
print()
print("Unique values in each column:\n",df_unclean.nunique())  ## checking for unique values in each column 
print()
print("Summary Statistics for Numerical Columns:\n",df_unclean.describe())  ## checking summary statistics for numerical columns

                             Task 02: Handling Missing values and Outliers:

In [2]:
df_clean = df_unclean.copy()  ## making a deep copy to work on

df_clean.drop(["car_ID", "Untitled"], axis = 1, inplace=True)  ## dropping unnecessary columns

symboling_mask = np.mean(df_clean["symboling"])  ## creating a mask value to fill null
df_clean["symboling"].fillna(symboling_mask, inplace=True)  ## filling nulls

price_mask = df_clean["price"].median()
df_clean["price"].fillna(price_mask, inplace=True)

fueltype_mask = df_clean["fueltype"].mode()[0]
df_clean["fueltype"].fillna(fueltype_mask, inplace=True)


# print(df_clean.isnull().sum())  ## confirming all non-null columns

## Identifying outliers in the price column 
Q1 = np.percentile(df_clean["price"], 25)
Q3 = np.percentile(df_clean["price"], 75)

IQR = Q3 - Q1  ## interquartile range

lower_whisker = Q1 - 1.5*(IQR)  ## lower bound

upper_whisker = Q3 + 1.5*(IQR)  ## upper bound

outliers = df_clean[(df_clean["price"] < lower_whisker) | (df_clean["price"] > upper_whisker)]  ## conditional formatting
price_outlier_count = outliers["price"].count()

print(f"Possible outliers in the price column are:\n{outliers}")

df_clean["price"] = np.log1p(df_clean["price"])  ## normalizing the extreme prizes 

Possible outliers in the price column are:
     Unnamed: 0  symboling                   CarName fueltype aspiration  \
15           15   0.000000                    bmw x4      gas        std   
16           16   0.000000                    bmw x5      gas        std   
17           17   0.000000                    bmw x3      gas        std   
47           47   0.000000                 jaguar xj      gas        std   
48           48   0.000000                 jaguar xf      gas        std   
67           67  -1.000000  buick electra 225 custom   diesel      turbo   
68           68  -1.000000  buick century luxus (sw)   diesel      turbo   
69           69   0.841463             buick century   diesel      turbo   
70           70  -1.000000             buick skyhawk   diesel      turbo   
71           71  -1.000000   buick opel isuzu deluxe      gas        std   
72           72   3.000000             buick skylark      gas        std   
73           73   0.841463     buick century 

                             Task 03: Data Cleaning and Preparation: 

In [7]:
# print(df_clean.head(25))  ## checking the carname column 

car_companies = pd.Series([car.split(" ")[0] for car in df_clean['CarName']], index = df_clean.index)

# df_clean.drop(columns= ['CarName'], axis = 1, inplace = True)

df_clean["car_company"] = car_companies  ## adding new column 'car_company'

## fixing some typos
df_clean["car_company"].replace({"vokswagen":"volkswagen", "toyouta":"toyota", "maxda":"mazda", "vw":"volkswagen", "porcshce": "porsche"}, inplace=True)

## counting the number of cars by each company
df_clean_company_pivot = df_clean.pivot_table(values="CarName", index="car_company", aggfunc="count").sort_values("CarName", ascending=False)
print(df_clean_company_pivot)

             CarName
car_company         
toyota            32
nissan            17
mazda             17
mitsubishi        13
honda             13
volkswagen        12
subaru            12
peugeot           11
volvo             11
dodge              9
buick              8
bmw                8
audi               7
plymouth           7
saab               6
porsche            5
isuzu              4
jaguar             3
alfa-romero        3
chevrolet          3
renault            2
Nissan             1
mercury            1


                             Task 04: Identifying Numerical Factors Impacting Car Prices 

In [4]:
numerical_columns = df_clean.select_dtypes(np.number)   ## extracting numerical columns 
correlation = numerical_columns.corr()                  ## finding correlation between numerical columns 

price_correlations = correlation["price"]               ## picking the subset,co-relation of price with other numerical columns 

## these are the correlation of other numerical columns with 'price'
print(f"Correlation coefficients with 'price':\n{price_correlations}")

print()

## these are the factors that have (absolute value of correlation > 0.4) have significant impact on car prices
significant_factors = price_correlations[np.abs(price_correlations) > 0.4]

print(f"Factors with significant impact on car prices:\n{significant_factors}")


Correlation coefficients with 'price':
Unnamed: 0         -0.085602
symboling          -0.022127
wheelbase           0.553529
carlength           0.683837
carwidth            0.702964
carheight           0.158797
curbweight          0.785020
enginesize          0.708287
boreratio           0.515868
stroke              0.116368
compressionratio    0.079953
horsepower          0.718771
peakrpm            -0.081336
citympg            -0.681929
highwaympg         -0.675603
price               1.000000
Name: price, dtype: float64

Factors with significant impact on car prices:
wheelbase     0.553529
carlength     0.683837
carwidth      0.702964
curbweight    0.785020
enginesize    0.708287
boreratio     0.515868
horsepower    0.718771
citympg      -0.681929
highwaympg   -0.675603
price         1.000000
Name: price, dtype: float64


                             Task 05: Identifying Categorical Factors Impacting Car Prices 

In [5]:
categorical_columns = df_clean.select_dtypes("object", "category")
unique_fueltypes = categorical_columns['fueltype'].unique()

for type in unique_fueltypes:
    print(type)
    subset = df_clean[df_clean["fueltype"] == type]
    price_description = subset["price"].describe()
    print(price_description)

gas
count    191.000000
mean       9.337526
std        0.441184
min        8.540714
25%        9.067428
50%        9.287209
75%        9.539529
max       10.629005
Name: price, dtype: float64
diesel
count    14.000000
mean      9.640715
std       0.541325
min       8.867850
25%       9.061825
50%       9.765265
75%      10.116378
max      10.360944
Name: price, dtype: float64


In [6]:
categorical_columns = df_clean.select_dtypes("object", "category")
# print(categorical_columns)

for column in categorical_columns.columns:
    unique_values = categorical_columns[column].unique()
    print(f"Impact of {column} on 'price':")

    for value in unique_values:
        subset = df_clean[df_clean[column] == value]
        price_description = subset["price"].describe()
        print(f"Category: {value}")
        print(price_description)
    print("\n")
        

Impact of CarName on 'price':
Category: alfa-romero giulia
count    1.000000
mean     9.510149
std           NaN
min      9.510149
25%      9.510149
50%      9.510149
75%      9.510149
max      9.510149
Name: price, dtype: float64
Category: alfa-romero stelvio
count    1.000000
mean     9.711176
std           NaN
min      9.711176
25%      9.711176
50%      9.711176
75%      9.711176
max      9.711176
Name: price, dtype: float64
Category: alfa-romero Quadrifoglio
count    1.000000
mean     9.711176
std           NaN
min      9.711176
25%      9.711176
50%      9.711176
75%      9.711176
max      9.711176
Name: price, dtype: float64
Category: audi 100 ls
count    1.000000
mean     9.543306
std           NaN
min      9.543306
25%      9.543306
50%      9.543306
75%      9.543306
max      9.543306
Name: price, dtype: float64
Category: audi 100ls
count    2.000000
mean     9.774547
std      0.010457
min      9.767152
25%      9.770849
50%      9.774547
75%      9.778244
max      9.781941
N